In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.stats import norm, pearsonr
from scipy.integrate import quad
import pickle
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [ ]:
from bayesee.evaluation import *
from bayesee.perception import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
n_location = 19
n_trial = 5000
n_init = 3

local_ecc = np.array([0, *[4,]*6, *[6.9295,]*6, *[8,]*6])

prior = np.array((0.5, *((0.5 / (n_location),) * (n_location))))
assert np.allclose(prior.sum(), 1.0)

e_prior = prior.copy()
assert np.allclose(e_prior.sum(), 1.0)

log_prior_ratio = np.log(e_prior / e_prior[0])
log_likelihood_ratio = np.zeros_like(prior)

model_target = np.random.choice(range(n_location+1), size=n_trial, p=prior)
model_response = np.zeros_like(model_target)

In [ ]:
def cost_function(x, array_d0, array_ed):
    # x[0]: d' peak
    # x[1]: d' fall-off

    cost = 0
    
    e_local_dp = x[0] * x[1] / (local_ecc + x[1])
    
    for d0 in array_d0:
        for ed in array_ed:
    
            local_dp = d0 * ed / (local_ecc + ed)
            
            for index_trial in range(n_trial):

                array_standard_normal = np.random.normal(size=(n_location,))
                
                log_likelihood_ratio[1:] = e_local_dp * (
                    array_standard_normal - e_local_dp / 2
                )

                if model_target[index_trial] > 0:
                    log_likelihood_ratio[model_target[index_trial]] += (
                        e_local_dp[model_target[index_trial] - 1]
                        * local_dp[model_target[index_trial] - 1]
                    )

                log_posterior_ratio = log_prior_ratio + log_likelihood_ratio
                model_response[index_trial] = np.argmax(log_posterior_ratio)
                
                if model_response[index_trial] == model_target[index_trial]:
                    cost -= 1

    return cost / (len(array_d0) * len(array_ed) * n_trial)

In [ ]:
opt_results = pd.DataFrame(columns=["dp0", "ed", "cost", "nfunc", "niter"])
bounds = [(1,9), (6.6, 6.6)]
array_d0 = [3, 4.5, 6, 7.5, 9]
array_ed = [1,3,5,7,9]

for index_init in range(n_init):
    init = [(bound[1] - bound[0]) * np.random.rand() + bound[0] for bound in bounds]
    
    results = minimize(
        cost_function,
        init,
        args=(array_d0, array_ed),
        bounds=bounds,
        method="Powell",
    )
    
    row_result = [ *results.x, results.fun, results.nfev, results.nit]    
    
    if index_init == 0:
        opt_results.loc[0] = row_result
    elif results.fun < opt_results.iloc[-1]["cost"]:
        opt_results.loc[len(opt_results)] = row_result
        
    print(f"{index_init+1}/{n_init}")

row_result = opt_results.iloc[-1].values
print(f"| {-row_result[-3]:.5f} | {row_result[0]:.3f} | {row_result[1]:.3f} |")